# Computational Methods in Economics

## Problem Set - Solving Systems of Linear Equations: Suggested Solutions

In [2]:
# Author: Alex Schmitt (schmitt@ifo.de), Christina Littlejohn (littlejohn@ifo.de)

import datetime
print('Last update: ' + str(datetime.datetime.today()))

Last update: 2018-11-15 11:11:46.215912


### Preliminaries

#### Import Modules

In [3]:
import numpy as np
import scipy.optimize
import scipy.linalg

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

# import sys
from importlib import reload

## Question 1 (A)

(a) Find the largest positive integer $n$ such that $2^n - 1$ is a DP number.

(b) Show that the distance between two adjacent DP numbers $y_1$ and $y_2$ with $y_1 < y_2$ that, 

\begin{equation}
   | y_2 - y_1 | = \epsilon_{DP} 2^{e(y_1) - 1023}. 
\end{equation}

Hint: Consider two cases for $e$ and $f$ that make two DP numbers adjacent.

(c) Show that the relativ absolute difference between a real number $z$ and the DP number closest to it, $z_{DP}$, is bounded above by half machine epsilon, i.e.

\begin{equation}
   \left| \frac{z_{DP} - z}{z} \right| \le \frac{1}{2} \epsilon_{DP}.
\end{equation}

Hint: Use the result from (b).

## Question 2 (N)

Write a function **backward_sub** that implements the backward-substitution algorithm to solve an upper triangular system of equations in Python. As a first step, derive an expression for $x_i$, analogous to the case of forward-substitution in the lecture.

In [4]:
def backward_sub(A, b):
    """
    Implements the backward-substitution algorithm to solve an upper triangular system of equations
    
    For doctest:
    
    >>> backward_sub( np.array([[1, 1], [0, 1]]), np.array([2, 1])  )
    array([ 1.,  1.])
    
    >>> backward_sub( np.array([[1, 2, 3], [0, 5, 7], [0, 0, 9]]), np.array([1, 2, 3])  )
    array([ 0.13333333, -0.06666667,  0.33333333])
    
     
    """
    ## check input: is A a square matrix?
    n, m = A.shape
    assert n == m, "A must be a square matrix"
    
    ## initialize solution vector
    x = np.zeros(n)
    
    ## fill solution vector using a for loop
    for i in range(n):
        
        ## compute sum on numerator of recursive rule
        summ = 0
        for j in range(i):
            summ += A[(n-1)-i, (n-1)-j] * x[(n-1)-j]
        
        ## use rule; NB: start at the last element in x!
        x[(n-1)-i] = (b[(n-1)-i] - summ) / A[(n-1)-i, (n-1)-i]
        
    return x        

In [5]:
## alternative: use vector multiplication for sum (instead of inner loop)

def backward_sub2(A, b):
    """
    Implements the backward-substitution algorithm to solve an upper triangular system of equations
    
    For doctest:
    
    >>> backward_sub( np.array([[1, 1], [0, 1]]), np.array([2, 1])  )
    array([ 1.,  1.])
    
    >>> backward_sub( np.array([[1, 2, 3], [0, 5, 7], [0, 0, 9]]), np.array([1, 2, 3])  )
    array([ 0.13333333, -0.06666667,  0.33333333])
    
    
    """
    ## check input: is A a square matrix?
    n, m = A.shape
    assert n == m, "A must be a square matrix"
    
    ## initialize solution vector
    x = np.zeros(n)
    
    ## fill solution vector using a for loop
    for i in range(n):
        
        ## compute sum on numerator of recursive rule 
        summ = A[(n-1)-i, (n-1) - (i-1):] @ x[(n-1) - (i-1):]
        
        ## use rule; NB: start at the last element in x!
        x[(n-1)-i] = (b[(n-1)-i] - summ) / A[(n-1)-i, (n-1)-i]
        
    return x
        

In [6]:
## test case
A = np.array([[1, 2, 3],
              [0, 5, 7],
              [0, 0, 9]])

b = np.array([1, 2, 3])

In [7]:
backward_sub(A, b)

array([ 0.13333333, -0.06666667,  0.33333333])

In [8]:
backward_sub2(A, b)

array([ 0.13333333, -0.06666667,  0.33333333])

In [9]:
## check our result
np.linalg.solve(A, b)

array([ 0.13333333, -0.06666667,  0.33333333])

In [10]:
## Alternative: use doctest
import doctest
doctest.testmod()

TestResults(failed=0, attempted=4)

## Question 3 (N)

Solve Exercise 2.2 in Miranda and Fackler. That is, find the solution to a SLE with

In [11]:
A = np.array([[54, 14, -11, 2], 
              [14, 50, -4, 29],
              [-11, -4, 55, 22],
              [2, 29, 22, 95]]
            )
b = np.array([1, 1, 1, 1])

numerically, using
a) LU factorization,
b) Gauss-Jacobi,
c) Gauss-Seidel.

For LU factorization, do not use the **linalg.solve** functions in Numpy or Scipy. However, you can use Scipy's **linalg.lu** function.

#### (a) LU Factorization

In [12]:
def forward_sub(A, b):
    """
    Implements the forward-substitution algorithm to solve a lower triangular system of equations
    """
    n, m = A.shape
    
    assert n == m, "A must be a square matrix"
    
    x = np.zeros(n)
    
    for i in range(n):
        
        summ = 0
        for j in range(i):
            summ += A[i, j] * x[j]
        
        x[i] = (b[i] - summ) / A[i, i]   
    
    return x

In [13]:
%%timeit -n1 -r1
P, L, U = scipy.linalg.lu(A)
y = forward_sub(L, b)
x = backward_sub(U, y)
print(x)


[ 0.01893441  0.01680508  0.02335523 -0.00041085]
1 loop, best of 1: 131 ms per loop


#### (b) Gauss-Jacobi

In [14]:
def gauss_jacobi(A, b, x0):
    """ 
    Implements the Gauss-Seidel method with a over-relaxation parameter
    """
    eps = 1
    tol = 1e-8
    it = 0
    maxit = 100

    x = x0
    Q = np.diag(np.diag(A))
    Q_inv = np.linalg.inv(Q)
    
    while eps > tol and it < maxit:
        it += 1 
        x_new = Q_inv @ b + ( np.eye(len(b)) - Q_inv @ A) @ x
        eps = np.linalg.norm(x_new - x)
        x = x_new
        
        
    return x, it 

In [15]:
%%timeit -n1 -r1
x0 = np.zeros(4)
gauss_jacobi(A, b, x0)
x, it = gauss_jacobi(A, b, x0)
print('Number of iterations = {}'.format(it))
print(x)

Number of iterations = 25
[ 0.01893441  0.01680507  0.02335523 -0.00041085]
1 loop, best of 1: 375 ms per loop


#### (c) Gauss-Seidel 

In [16]:
def gauss_seidel(A, b, x0):
    """ 
    Implements the Gauss-Seidel method with a over-relaxation parameter
    """
    eps = 1
    tol = 1e-8
    it = 0
    maxit = 100

    x = x0
    Q = np.triu(A)
    Q_inv = np.linalg.inv(Q)
    
    while eps > tol and it < maxit:
        it += 1 
        x_new = Q_inv @ b + ( np.eye(len(b)) - Q_inv @ A) @ x
        eps = np.linalg.norm(x_new - x)
        x = x_new
        
    return x, it  

In [17]:
%%timeit -n1 -r1
x0 = np.zeros(4)
gauss_seidel(A, b, x0) 
x, it = gauss_seidel(A, b, x0)
print('Number of iterations = {}'.format(it))
print(x)

Number of iterations = 14
[ 0.01893441  0.01680508  0.02335523 -0.00041085]
1 loop, best of 1: 120 ms per loop


## Question 4 (N)


From Judd (1998), chapter 3. Suppose that demand for good $i$ is 

\begin{equation}
    d_i(p) = a_i \sum_{j \neq i} p_j - b_ip_i + c_i,\ \ i = 1, ..., n
\end{equation}

where $a_i > b_i > 0$, and that supply is 

\begin{equation}
    s_i(p) = A_i + B_i p_i, \ \ i = 1, ..., n 
\end{equation}

where $B_i > 0$. Write a program to solve for equilibrium with sensible choices of $a_i$, $b_i$, $A_i$ and $B_i$. That means read in the parameters, check that they are consistent with commonsense economics (downward sloping demand curves, increasing supply curves and concave utility) and output the equilibrium prices and outputs. Your code should work for any number $n \ge 1$.

In [18]:
def find_eq(a,b,c,A,B):
    """ 
    Takes in the parameters for a, b, c, A, B as flat numpy arrays. Solves for equilibrium
    price and quantity vectors given sensible choices of a_i, b_i, A_i, B_i.
    
    Returns the price and quantity vectors as two flat numpy arrays.
    """
    
    # get length of the array
    m, = b.shape
    
    # check that b > 0 and a > b
    assert b.min() > 0,    'All values in b must be greater than 0'
        
    for i in range(m):    
        if a[i] < b[i]:
            print('At position {} the a !> b'.format(i))
        else: continue
            
    # check that B > 0 
    assert B.min() > 0,    'All values in B must be greater than 0'
    
    # create matrix M populated with values from (-b_i - B_i) and a
    M = np.zeros((m,m))
    for i in range(m):
        for j in range(m):
            if i == j:
                M[i,j] = -b[i] - B[i]
            else:
                M[i,j] = a[i]
          
    # create array x with values A_i - c_i
    x = A - c
 
    # use gaussian to compute p vector
    p0 = np.zeros(m)
    p, it = gauss_jacobi(M, x, p0)
    
    # compute q vector with A + B*p
    q = A + (B * p)
    
    return p, q

In [19]:
a = np.array([.3,.4,.4,1,.8])
b = np.array([.2,.2,.3,.4,.5])
c = np.array([1,1,1,1,1])
A = np.array([-5,-4,-5,6,7])
B = np.array([5,6,1,8,7])


p_eq, q_eq = find_eq(a,b,c,A,B)
print('p_eq = ', p_eq) 
print('q_eq = ', q_eq)

p_eq =  [ 1.5942729   1.31686888  5.70078507  0.44982303  0.16658665]
q_eq =  [ 2.97136451  3.90121328  0.70078507  9.59858428  8.16610657]


## Question 5

This question follows the example for the use of pivoting in Miranda and Fackler, section 2.3.

(a) Start with matrix 

\begin{equation}
    A = \left[\begin{array}{cc}
     -1\text{e-20} & 1 \\
     1 & 1
     \end{array}
     \right]
\end{equation}
 and vector 
\begin{equation}
    b = \left[\begin{array}{c}
     2 \\
     1 
     \end{array}
     \right]
\end{equation} 

Decompose $A$ analytically in matrices $L$ and $U$ using Gaussian elimination and confirm numerically that $A = L U$.

(b) Solve the system $Ax = b$ analytically using forward and backward substitution on $L$ and $U$ found in (a), and show that

\begin{equation}
    x_1 = \frac{1\text{e+20}}{1\text{e+20} + 1} \approx 1,\quad x_2 = \frac{1\text{e+20} + 2}{1\text{e+20} + 1} \approx 1
\end{equation} 

(c) Solve $Ax = b$ numerically using forward and backward substitution and compare the results. 

(d) Perform *pivoting* on the system: move greater elements to the diagonal by interchanging rows. In the system above, that gives matrix $\hat{A}$ and vector $\hat{b}$:

\begin{equation}
    \hat{A} = \left[\begin{array}{cc}
     1 & 1 \\
     -1\text{e-20} & 1
     \end{array}
     \right]
\end{equation}
 and vector 
\begin{equation}
    \hat{b} = \left[\begin{array}{c}
     1 \\
     2 
     \end{array}
     \right]
\end{equation} 

Verify that the system $\hat{A}x = \hat{b}$ has the same solution as the original system.

(e) As before, decompose $\hat{A}$ analytically in matrices $\hat{L}$ and $\hat{U}$ using Gaussian elimination and confirm numerically that $\hat{A} = \hat{L} \hat{U}$.

(f) Solve $\hat{A}x = \hat{b}$ numerically and verify that the result is close to the analytical solution found in (b).

You will need your implementation of backward substitution from question 2, as well as forward substitution as seen in the lecture:

In [20]:
def forward_sub(A, b):
    """
    Implements the forward-substitution algorithm to solve a lower triangular system of equations
    """
    n, m = A.shape
    
    assert n == m, "A must be a square matrix"
    
    x = np.zeros(n)
    for i in range(n):
        
        summ = 0
        for j in range(i):
            summ += A[i, j] * x[j]
        
        x[i] = (b[i] - summ) / A[i, i]   
    
    return x

#### Answer

##### (a)

In [21]:
M = 1e+20
A = np.array([[-M**(-1), 1],
            [1, 1]])
b = np.array([1, 2])

print(A)

[[ -1.00000000e-20   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00]]


In [22]:
## results from analytical decomposition
L = np.array([[1, 0],
            [-M, 1]])
U = np.array([[-M**(-1), 1],
            [0, M + 1]])

print( L @ U )

[[ -1.00000000e-20   1.00000000e+00]
 [  1.00000000e+00   0.00000000e+00]]


##### (c)

In [23]:
## solve system numerically using backward and forward substitution
print( backward_sub(U, forward_sub(L, b)) )

[-0.  1.]


##### (e) 

In [27]:
## system with pivoting
A_hat = np.array([[1, 1],
            [-M**(-1), 1]])
b_hat = np.array([2, 1])

print(A_hat)

[[  1.00000000e+00   1.00000000e+00]
 [ -1.00000000e-20   1.00000000e+00]]


In [28]:
L_hat = np.array([[1, 0],
            [-M**(-1), 1]])
U_hat = np.array([[1, 1],
            [0, M**(-1) + 1]])
print( L_hat @ U_hat )

[[  1.00000000e+00   1.00000000e+00]
 [ -1.00000000e-20   1.00000000e+00]]


##### (f)

In [29]:
## solve system numerically using backward and forward substitution
backward_sub(U_hat, forward_sub(L_hat, b_hat))

array([ 1.,  1.])